In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
from torch.optim import Adam



In [2]:
#loading dataset
dataset= load_dataset("tweet_eval", "irony")

tokenizer = BertTokenizer.from_pretrained ('bert-base-uncased')

class SarcasmDataset (torch.utils.data.Dataset):
    def __init__ (self, split):
        self.data=dataset[split]
        self.texts = self.data[ 'text']
        self.labels= self.data['label']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text=self.texts[idx]
        label=self.labels[idx]
        encoding=tokenizer(text, truncation=True, padding ='max_length', max_length=64, return_tensors='pt')

        return {key: val.squeeze(0) for key,val in encoding.items()}, torch.tensor(label)


train_data=SarcasmDataset('train')
val_data= SarcasmDataset('validation')
train_loader = DataLoader(train_data, batch_size=16 , shuffle =True)
val_loader = DataLoader(val_data, batch_size=32)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/955 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
len(train_data), len(val_data), type(train_data), type(train_loader), len(train_loader)

In [10]:
class SarcasmClassifier (nn.Module):
    def __init__(self):
        super().__init__()
        self.bert =BertModel.from_pretrained ('bert-base-uncased')
        self.classifier=nn.Linear(self.bert.config.hidden_size, 1)
        self.sigmoid=nn.Sigmoid()
    def forward(self, input_ids, attention_mask):
        outputs= self.bert(input_ids= input_ids, attention_mask=attention_mask)
        pool_output=outputs.pooler_output
        logits=self.classifier(pool_output)
        return self.sigmoid(logits).squeeze(-1)


model= SarcasmClassifier()


device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=2e-5)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [12]:


for epoch in range(3):
    print(f"Current epoch {epoch}")

    model.train()
    total_loss=0
    for batch, labels in train_loader:
        input_ids=batch['input_ids' ].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels=labels.float().to(device)
        optimizer.zero_grad()
        outputs=model(input_ids, attention_mask)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss+=loss.item()

        print(f"train loss: {total_loss/len(train_loader):.4f}")


    model.eval()
    total_correct = 0
    total =0
    with torch.no_grad():
        for batch, labels in val_loader:
            input_ids= batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels= labels.to(device)
            outputs= model(input_ids, attention_mask)
            preds=(outputs > 0.5).long()
            total_correct+= (preds==labels).sum().item()
            total+=labels.size(0)

    print(f"Validation accuracy: {total_correct/total: .4f}")


Current epoch 0
train loss: 0.0031
train loss: 0.0058
train loss: 0.0086
train loss: 0.0114
train loss: 0.0143
train loss: 0.0173
train loss: 0.0204
train loss: 0.0226
train loss: 0.0243
train loss: 0.0272
train loss: 0.0295
train loss: 0.0313
train loss: 0.0329
train loss: 0.0350
train loss: 0.0372
train loss: 0.0390
train loss: 0.0424
train loss: 0.0461
train loss: 0.0506
train loss: 0.0524
train loss: 0.0550
train loss: 0.0579
train loss: 0.0606
train loss: 0.0639
train loss: 0.0655
train loss: 0.0672
train loss: 0.0699
train loss: 0.0730
train loss: 0.0775
train loss: 0.0795
train loss: 0.0819
train loss: 0.0843
train loss: 0.0871
train loss: 0.0882
train loss: 0.0909
train loss: 0.0937
train loss: 0.0976
train loss: 0.1018
train loss: 0.1042
train loss: 0.1077
train loss: 0.1092
train loss: 0.1123
train loss: 0.1140
train loss: 0.1181
train loss: 0.1198
train loss: 0.1237
train loss: 0.1260
train loss: 0.1279
train loss: 0.1298
train loss: 0.1314
train loss: 0.1336
train loss: 0.1

In [13]:
torch.save(model.state_dict(), "sarcasm_model.pt")


In [87]:
def custom_text(texts: str):
  texts=[texts]
  inputs=tokenizer(texts, padding=True, truncation=True, return_tensors='pt', return_token_type_ids=False)
  inputs={k: v.to(device) for k,v in inputs.items()}

  model.eval()
  model.to(device)
  with torch.no_grad():
    proba=model(**inputs)
  preds=(proba > 0.5).long()

  return [(text,pred.item(), prob.item()) for text, pred, prob in zip(texts,preds, proba)]




In [88]:

def print_result(texts: str):
  results= custom_text(texts)
  for text, pred, prob in results:
    label= "Bazinga" if pred==1 else "Not Funny at all."
    print(f'Text: {text} \n  {label} (probability: {prob: .3f})\n')


In [89]:
print_result("Just what I needed at this hour! a flat tire. Amazing")

Text: Just what I needed at this hour! a flat tire. Amazing 
  Bazinga (probability:  0.771)

